# Reading data from files

In [33]:
import pandas as pd
import matplotlib.pyplot as plt

In [31]:
data_X = pd.read_csv('C:\\Users\\Hamza Ali Joiyah\\Desktop\\Data Science Project\\UCI HAR Dataset\\UCI HAR Dataset\\train\\X_train.txt', sep = "delimiter", header = None)

C:\Users\Hamza Ali Joiyah\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [30]:
data_y = pd.read_csv('C:\\Users\\Hamza Ali Joiyah\\Desktop\\Data Science Project\\UCI HAR Dataset\\UCI HAR Dataset\\train\\y_train.txt', sep = "delimiter", header = None)

C:\Users\Hamza Ali Joiyah\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [34]:
data_y

,0
0,5
1,5
2,5
3,5
4,5
5,5
6,5
7,5
8,5
9,5


In [35]:
subject_train = pd.read_csv('C:\\Users\\Hamza Ali Joiyah\\Desktop\\Data Science Project\\UCI HAR Dataset\\UCI HAR Dataset\\train\\subject_train.txt', sep = "delimiter", header = None)

C:\Users\Hamza Ali Joiyah\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [36]:
subject_train

,0
0,1
1,1
2,1
3,1
4,1
5,1
6,1
7,1
8,1
9,1
